This notebook is meant for our lecture 4 breakout session.  I've put together some questions to guide your [here](https://itempool.com/MIT-Robotic-Manipulation/c/yzVbTp9zFc_).  The points are not real, and will not contribute to your grade!

# Notebook setup

The following cell will:
- on Colab (only), install Drake to `/opt/drake`, install Drake's prerequisites via `apt`, and add pydrake to `sys.path`.  This will take approximately two minutes on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  If you navigate between notebooks using Colab's "File->Open" menu, then you can avoid provisioning a separate machine for each notebook.
- launch a server for our 3D visualizer (MeshCat) that will be used for the remainder of this notebook.

You will need to rerun this cell if you restart the kernel, but it should be fast because the machine will already have drake installed.

In [ ]:
import importlib
import sys
from urllib.request import urlretrieve

# Install drake.
if 'google.colab' in sys.modules and importlib.util.find_spec('pydrake') is None:
  version='20200909'
  build='nightly'
  urlretrieve(f"https://drake-packages.csail.mit.edu/drake/{build}/drake-{version}/setup_drake_colab.py",
              "setup_drake_colab.py")
  from setup_drake_colab import setup_drake
  setup_drake(version=version, build=build)

# Install pyngrok.
server_args = []
if 'google.colab' in sys.modules:
  !pip install pyngrok
  server_args = ['--ngrok_http_tunnel']

# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

# Determine if this notebook is currently running as a notebook or a unit test.
from IPython import get_ipython
running_as_notebook = get_ipython() and hasattr(get_ipython(), 'kernel')

# Let's do all of our imports here, too.
import numpy as np
import altair as alt
import pydot
from IPython.display import display, SVG
from ipywidgets import Text, Textarea, Layout

from pydrake.all import (
    AddMultibodyPlantSceneGraph, AngleAxis, BasicVector, ConnectMeshcatVisualizer, 
    DiagramBuilder, FindResourceOrThrow, Integrator, JacobianWrtVariable, 
    LeafSystem, MultibodyPlant, MultibodyPositionToGeometryPose, Parser, 
    PiecewisePolynomial, PiecewiseQuaternionSlerp, Quaternion, RigidTransform, 
    RollPitchYaw, RotationMatrix, SceneGraph, Simulator, TrajectorySource,
    DifferentialInverseKinematicsParameters, DifferentialInverseKinematicsIntegrator,
    set_log_level
)
from pydrake.systems.jupyter_widgets import PoseSliders, WidgetSystem
from pydrake.examples.manipulation_station import ManipulationStation

# TODO(russt): upstream this to drake
import numpy as np
from functools import partial

from IPython.display import display
from ipywidgets import FloatSlider, Layout, ToggleButton, ToggleButtons

from pydrake.all import JointIndex

def MakeJointSlidersThatPublishOnCallback(
    plant, publishing_system, root_context, my_callback=None, 
    lower_limit=-10., upper_limit=10., resolution=0.01, length=200):
    """ 
    Creates an ipywidget slider for each joint in the plant.  Unlike the 
    JointSliders System, we do not expect this to be used in a Simulator.  It 
    simply updates the context and calls Publish directly from the slider 
    callback.
    
    Args:
        plant:        A MultibodyPlant.
        publishing_system: The System whos Publish method will be called.  Can 
                           be the entire Diagram, but can also be a subsystem.
        root_context: A mutable root Context of the Diagram containing both the 
                      ``plant`` and the ``publishing_system``; we will extract 
                      the subcontext's using `GetMyContextFromRoot`.
        my_callback:  An optional additional callback function that will be 
                      called when the widgets update, using  
                      ``my_callback(plant_context)``.
        lower_limit:  A scalar or vector of length robot.num_positions().
                      The lower limit of the slider will be the maximum
                      value of this number and any limit specified in the
                      Joint.
        upper_limit:  A scalar or vector of length robot.num_positions().
                      The upper limit of the slider will be the minimum
                      value of this number and any limit specified in the
                      Joint.
        resolution:   A scalar or vector of length robot.num_positions()
                      that specifies the discretization of the slider.
        length:       The length of the sliders.
    
    Returns:
        A list of the slider widget objects that are created.

    Note: Some publishers (like MeshcatVisualizer) use an initialization event 
    to "load" the geometry.  You should call that *before* calling this method 
    (e.g. with `meshcat.load()`).
    """

    def _reshape(x, num):
        x = np.array(x)
        assert len(x.shape) <= 1
        return np.array(x) * np.ones(num)

    lower_limit = _reshape(lower_limit, plant.num_positions())
    upper_limit = _reshape(upper_limit, plant.num_positions())
    resolution = _reshape(resolution, plant.num_positions())

    publishing_system_context = publishing_system.GetMyContextFromRoot(root_context)
    plant_context = plant.GetMyContextFromRoot(root_context)
    positions = plant.GetPositions(plant_context)

    # Publish once immediately.
    publishing_system.Publish(publishing_system_context)
    if my_callback:
        my_callback(plant_context)

    def _slider_callback(change, index):
        positions[index] = change.new
        plant.SetPositions(plant_context, positions)
        publishing_system.Publish(publishing_system_context)
        if my_callback:
            my_callback(plant_context)

    slider_widgets = []
    k = 0
    for i in range(0, plant.num_joints()):
        joint = plant.get_joint(JointIndex(i))
        low = joint.position_lower_limits()
        upp = joint.position_upper_limits()
        for j in range(0, joint.num_positions()):
            index = joint.position_start() + j
            slider = FloatSlider(
                value=positions[index],
                min=max(low[j], lower_limit[k]),
                max=min(upp[j], upper_limit[k]),
                step=resolution[k],
                continuous_update=True,
                description=joint.name(),
                style={'description_width': 'initial'},
                layout=Layout(width=f"'{length}'"))
            slider.observe(partial(_slider_callback, index=index), names='value')
            display(slider)
            slider_widgets.append(slider)
            k += 1

    return slider_widgets

# Kinematic Jacobians for pick and place

Let's set up the same iiwa + wsg example, with sliders (but without the frames), that we used above.  But this time I'll display the value of the Jacobian $J^G(q)$.

In [ ]:
def pick_and_place_jacobians_example():
    builder = DiagramBuilder()

    # TODO: Replace this with a simple model directive of iiwa+wsg (no clutter bins)
    station = builder.AddSystem(ManipulationStation())
    station.SetupClutterClearingStation()
    station.Finalize()

    frames_to_draw = {"gripper": {"body"}}
    meshcat = ConnectMeshcatVisualizer(builder,
        station.get_scene_graph(),
        output_port=station.GetOutputPort("pose_bundle"),
        delete_prefix_on_load=False,
        zmq_url=zmq_url,
        frames_to_draw=frames_to_draw,
        axis_length=0.2,
        axis_radius=0.005)

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()

    jacobian = Textarea(value="", description="J_G: ", layout={'width':'60%','height':'100px'}, style={'description_width':'initial'})
    sigma = Text(value="", description="smallest singular value(J_G): ", layout={'width':'60%'}, style={'description_width':'initial'})
    plant = station.get_multibody_plant()

    G = plant.GetBodyByName("body").body_frame()
    W = plant.world_frame()
    def pose_callback(context):
        J_G = plant.CalcJacobianSpatialVelocity(context, JacobianWrtVariable.kQDot, G, [0,0,0], W, W)   ## This is the important line
        jacobian.value = np.array2string(J_G, formatter={'float': lambda x: "{:5.2f}".format(x)})
        sigma.value = str(np.min(np.linalg.svd(J_G, compute_uv=False)))

    meshcat.load()

    # If you want to set the initial positions manually, use this:
    # plant.SetPositions(plant.GetMyContextFromRoot(context),
    #                   plant.GetModelInstanceByName("iiwa"),
    #                   [0, 0, 0, 0, 0, 0, 0])
    # The default positions for this "clutter clearing" setup are:
    #                   [-1.57, 0.1, 0, -1.2, 0, 1.6, 0]

    MakeJointSlidersThatPublishOnCallback(station.get_multibody_plant(), meshcat, context, my_callback=pose_callback)
    display(jacobian)
    display(sigma)

pick_and_place_jacobians_example()


# Teleop example from lecture 1

Here is the example I gave you before.  Can you drive the Jacobian into singularity (like you might have done before).  Does it make more sense now?

If you get stuck and need to restart, just press the "Stop simulation" button, then run the cell again.


In [ ]:
def teleop_example():
    set_log_level("warn")
    builder = DiagramBuilder()

    station = builder.AddSystem(ManipulationStation())

    station.SetupClutterClearingStation()
    station.Finalize()

    frames_to_draw = {"gripper": {"body"}}
    meshcat = ConnectMeshcatVisualizer(builder,
        station.get_scene_graph(),
        output_port=station.GetOutputPort("pose_bundle"),
        delete_prefix_on_load=False,
        zmq_url=zmq_url,
        frames_to_draw=frames_to_draw,
        axis_length=0.2,
        axis_radius=0.005)

    robot = station.get_controller_plant()
    params = DifferentialInverseKinematicsParameters(robot.num_positions(),
                                                    robot.num_velocities())

    time_step = 0.005
    params.set_timestep(time_step)
    # True velocity limits for the IIWA14 (in rad, rounded down to the first
    # decimal)
    iiwa14_velocity_limits = np.array([1.4, 1.4, 1.7, 1.3, 2.2, 2.3, 2.3])
    params.set_joint_velocity_limits((-iiwa14_velocity_limits,
                                    iiwa14_velocity_limits))
    differential_ik = builder.AddSystem(DifferentialInverseKinematicsIntegrator(
        robot, robot.GetFrameByName("iiwa_link_7"), time_step, params))
    builder.Connect(differential_ik.get_output_port(),
                    station.GetInputPort("iiwa_position"))

    teleop = builder.AddSystem(PoseSliders(
        min_range = PoseSliders.MinRange(roll=0, pitch=-0.5, yaw=-np.pi, 
                                        x=-0.6, y=-0.8, z=0.0),
        max_range = PoseSliders.MaxRange(roll=2*np.pi, pitch=np.pi, yaw=np.pi,
                                        x=0.8, y=0.3, z=1.1)
    ))
    builder.Connect(teleop.get_output_port(0), 
                    differential_ik.get_input_port())
    wsg_buttons = ToggleButtons(value=0.107, description="SchunkWsg", 
                                options=[('Open', 0.107), ('Close', 0.002)])
    wsg_teleop = builder.AddSystem(WidgetSystem([wsg_buttons]))
    builder.Connect(wsg_teleop.get_output_port(0),
                    station.GetInputPort("wsg_position"))

    diagram = builder.Build()
    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()

    station_context = station.GetMyMutableContextFromRoot(context)

    q0 = station.GetOutputPort("iiwa_position_measured").Eval(
        station_context)
    differential_ik.get_mutable_parameters().set_nominal_joint_position(q0)
    diff_ik_context = differential_ik.GetMyMutableContextFromRoot(context)
    differential_ik.SetPositions(diff_ik_context, q0)
    teleop.SetPose(differential_ik.ForwardKinematics(diff_ik_context))

    if running_as_notebook:  # Then we're not just running as a test on CI.
        simulator.set_target_realtime_rate(1.0)

        stop_button = ToggleButton(value=False, description='Stop Simulation')
        display(stop_button)
        while not stop_button.value:
            simulator.AdvanceTo(simulator.get_context().get_time() + 2.0)
        stop_button.value = False
    
    else:
        simulator.AdvanceTo(0.1)

teleop_example()

# Our first end-effector "controller"

Let's use the pseudo-inverse of the Jacobian to drive the robot around.  To do that, we'll write a very simple system that looks at the current value of $q$, computes $[J^G]^+$, and uses it to command a constant spatial velocity, $V^G$.

We'll only run this controller for a short duration.  Constant spatial velocities aren't something that makes sense for a longer simulation!

Make sure you try changing $V^G$, and understand how the command relates to the motion of the robot.

In [ ]:
# We can write a new System by deriving from the LeafSystem class.
# There is a little bit of boiler plate, but hopefully this example makes sense.
class PseudoInverseController(LeafSystem):
    def __init__(self, plant):
        LeafSystem.__init__(self)
        self._plant = plant
        self._plant_context = plant.CreateDefaultContext()
        self._iiwa = plant.GetModelInstanceByName("iiwa")
        self._G = plant.GetBodyByName("body").body_frame()
        self._W = plant.world_frame()

        self.DeclareVectorInputPort("iiwa_position", BasicVector(7))
        self.DeclareVectorOutputPort("iiwa_velocity", BasicVector(7), 
                                     self.CalcOutput)

    def CalcOutput(self, context, output):
        q = self.get_input_port().Eval(context)
        self._plant.SetPositions(self._plant_context, self._iiwa, q)
        J_G = self._plant.CalcJacobianSpatialVelocity(
            self._plant_context, JacobianWrtVariable.kQDot, 
            self._G, [0,0,0], self._W, self._W)
        J_G = J_G[:,0:7] # Ignore gripper terms
        
        V_G_desired = np.array([0,    # rotation about x
                                -.1,  # rotation about y
                                0,    # rotation about z
                                0,    # x
                                -.05, # y
                                -.1]) # z
        v = np.linalg.pinv(J_G).dot(V_G_desired)
        output.SetFromVector(v)

        
def jacobian_controller_example():
    builder = DiagramBuilder()

    station = builder.AddSystem(ManipulationStation())
    station.SetupClutterClearingStation()
    station.Finalize()

    controller = builder.AddSystem(PseudoInverseController(
        station.get_multibody_plant()))
    integrator = builder.AddSystem(Integrator(7))

    builder.Connect(controller.get_output_port(), 
                    integrator.get_input_port())
    builder.Connect(integrator.get_output_port(),
                    station.GetInputPort("iiwa_position"))
    builder.Connect(station.GetOutputPort("iiwa_position_measured"),
                    controller.get_input_port())

    frames_to_draw = {"gripper": {"body"}}
    meshcat = ConnectMeshcatVisualizer(builder,
        station.get_scene_graph(),
        output_port=station.GetOutputPort("pose_bundle"),
        delete_prefix_on_load=False,
        zmq_url=zmq_url,
        frames_to_draw=frames_to_draw,
        axis_length=0.2,
        axis_radius=0.005)

    diagram = builder.Build()
    display(SVG(pydot.graph_from_dot_data(diagram.GetGraphvizString(max_depth=1))[0].create_svg()))
    
    simulator = Simulator(diagram)
    station_context = station.GetMyContextFromRoot(simulator.get_mutable_context())
    station.GetInputPort("iiwa_feedforward_torque").FixValue(station_context, np.zeros((7,1)))
    station.GetInputPort("wsg_position").FixValue(station_context, [0.1])
    # TODO(russt): Add this missing python binding
    #integrator.set_integral_value(
    #    integrator.GetMyContextFromRoot(simulator.get_mutable_context()), 
    #        station.GetIiwaPosition(station_context))
    integrator.GetMyContextFromRoot(simulator.get_mutable_context()).get_mutable_continuous_state_vector().SetFromVector(station.GetIiwaPosition(station_context))

    simulator.set_target_realtime_rate(1.0)
    simulator.AdvanceTo(5.0)

jacobian_controller_example()